# The Spinning Effective One-Body Hamiltonian

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body Hamiltonian as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.

### Throughout this module, we will refer to [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010 and [Barausse and Buonanno (2011)](https://arxiv.org/abs/1107.2904) as BB2011.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows:

0. [Introduction](#intro): The Physical System of Interest
1. [Step 1](#Hreal): The Real Hamiltonian $H_{\rm real}$
1. [Step 2](#Heff): The Effective Hamiltonian $H_{\rm eff}$
1. [Step 3](#Heff_terms): Terms of $H_{\rm eff}$  
    a. [Step 3a](#Hs): Leading Order Spin Effects $H_{\rm S}$  
    b. [Step 3b](#Hns): The Nonspinning Hamiltonian $H_{\rm NS}$  
    c. [Step 3c](#Hd): The Quadrupole Deformation $H_{\rm D}$
1. [Step 4](#Hso): The Spin-Orbit Term $H_{\rm SO}$  
    a. [Step 4a](#HsoTerm1): $H_{\rm SO}$ Term 1  
    a. [Step 4b](#HsoTerm2coeff): $H_{\rm SO}$ Term 2 Coefficient  
    a. [Step 4c](#HsoTerm2): $H_{\rm SO}$ Term 2
1. [Step 5](#Hss): The Spin-Spin Term $H_{\rm SS}$
1. [Step 6](#HsoHsscommon): Terms Common to $H_{\rm SO}$ and $H_{\rm SS}$
1. [Step N-5](#momentum_terms): The Momentum Terms
1. [Step N-4](#alphabetagamma): The Metric-Dependent Terms $\alpha$, $\beta^{i}$, and $\gamma^{ij}$
1. [Step N-3](#Hreal_spin_combos): $H_{\rm real}$ Spin Combinations ${\bf S}^{*}$ and ${\bf S}_{\rm Kerr}$
1. [Step N-2](#Heff_spin_combos): $H_{\rm eff}$ Spin Combinations $\boldsymbol{\sigma}$ and $\boldsymbol{\sigma}^{*}$
1. [Step N-1](#mass_terms): Mass Terms $\mu$, $M$, and $\eta$
1. [Step N](#metric): The Metric $g$ of Choice
1. [Notes](#notes)
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system.  The spinning effective one-body ("SEOB") Hamiltonian [$H_{\rm real}$](#Hreal) describes the dynamics of this system; we will define [$H_{\rm real}$](#Hreal) as in BB2010 Section VE.  There, [$H_{\rm real}$](#Hreal) is canonically transformed and mapped to an effective Hamiltonian [$H_{\rm eff}$](#Heff) describing a test particle of mass [$\mu$](#mass_terms) and spin [${\bf S}_{\rm Kerr}$](#Hreal_spin_combos).  Here we seek to break up [$H_{\rm real}$](#Hreal) and document the terms in such a way that the resulting Python code can be used to numerically evaluate [$H_{\rm real}$](#Hreal).  

We will use Greek letters to denote spacetime tensor indicies, and spacial tensor indicies with lowercase Latin letters.

<a id='Hreal'></a>

# Step 1: The real Hamiltonian $H_{\textrm{real}}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

The SEOB Hamiltonian $H_{\rm real}$ is given by Equation (5.69) of BB2010:

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \eta \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) }.
\end{equation}

Here [$H_{\rm eff}$](#Heff) is an *effective* Hamiltonian and [$M$](#mass_terms), [$\mu$](#mass_terms), and [$\eta$](#mass_terms) are constants determined by $m_{1}$ and $m_{2}$.

<font color='red'>The code in LALSimIMRSpinEOBHamiltonianPrec.c claims the origin of $H_{\textrm{real}}$ is Equation (2) of [Taracchini, Pan, Buonanno, et al](https://arxiv.org/abs/1202.0790):

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \nu \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) } - M
\end{equation}

where $\nu = \frac{\mu}{M}$.  Double-check this!</font>

In [1]:
%%writefile SEOBNR/Hamiltonian.txt
Hreal = M * sp.sqrt(1 + 2*eta*(Heff/mu - 1))

Overwriting SEOBNR/Hamiltonian.txt


<a id='Heff'></a>

# Step 2: The Effective Hamiltonian $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff}$$

The effective Hamiltonian $H_{\rm eff}$ is given by Equation (5.70) of BB2010.  We combine the notation of BB2010 Equations (5.70) and (5.43) with our notation [$H_{\rm D}$](#Hd) to write

\begin{equation}
    H_{\rm eff} = H_{\rm S} + H_{\rm NS} \underbrace{ -\frac{ \mu }{ 2 M r^{3} } \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}}.
\end{equation}

Here [$H_{\rm S}$](#Hs) denotes leading order effects of spin-spin and spin-orbit coupling, [$H_{\rm NS}$](#Hns) is the Hamiltonian for a nonspinning test particle, and [$H_{\rm D}$](#Hd) describes quadrupole deformation of the coupling of the particle's spin with itself to leading order.

In [2]:
%%writefile -a SEOBNR/Hamiltonian.txt

Heff = Hs + Hns + Hd

Appending to SEOBNR/Hamiltonian.txt


<a id='Heff_terms'></a>

# Step 3: Terms of $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff_terms}$$

In this step, we break down each of the terms [$H_{\rm S}$](#Hs), [$H_{\rm NS}$](#Hns), and [$H_{\rm D}$](#Hd) in [$H_{\rm eff}$](#Heff).

<a id='Hs'></a>

## Step 3a: Leading Order Spin Effects $H_{\rm S}$ \[Back to [top](#toc)\]
$$\label{Hs}$$

From BB2010 Equation (4.17),

\begin{equation}
    H_{\rm S} = H_{\rm SO} + H_{\rm SS}
\end{equation}

where [$H_{\rm SO}$](#Hso) includes spin-orbit terms and [$H_{\rm SS}$](#Hss) includes spin-spin terms.

In [3]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hs = Hso + Hss

Appending to SEOBNR/Hamiltonian.txt


<a id='Hns'></a>

## Step 3b: The Nonspinning Hamiltonian $H_{\rm NS}$ \[Back to [top](#toc)\]
$$\label{Hns}$$

From BB2010 Equation (5.43),

\begin{equation}
    H_{\rm NS} = \underbrace{ \beta^{i} p_{i} }_{\beta p\ {\rm sum}} + \alpha \sqrt{ \smash[b]{\underbrace{ \mu^{2} + \gamma^{ij} p_{i} p_{j} + {\cal Q}_{4} \left( p \right) }_{H_{\rm NS}\ {\rm radicand}}}}
\end{equation}

For further magic, see [$\beta p\ {\rm sum}$](#betapsum), [$\alpha$](#alpha), and [$H_{\rm NS}\ {\rm radicand}$](#Hnsradicand). 

In [4]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hns = betapsum + alpha*sp.sqrt(Hnsradicand)

Appending to SEOBNR/Hamiltonian.txt


<a id='Hd'></a>

## Step 3c: The Quadrupole Deformation $H_{\rm D}$ \[Back to [top](#toc)\]
$$\label{Hd}$$

We break up $H_{\rm D}$ as follows:

\begin{equation}
    H_{\rm D} = \underbrace{ -\frac{ \mu }{ 2 M r^{3} } }_{H_{\rm D}\ {\rm coefficient}} \underbrace{ \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}\ {\rm sum}}
\end{equation}

For further dissection, see [$H_{\rm D}\ {\rm coefficient}$](#Hdcoeff) and [$H_{\rm D}\ {\rm sum}$](#Hdsum).

In [5]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hd = Hdcoeff*Hdsum

Appending to SEOBNR/Hamiltonian.txt


<a id='Hso'></a>

# Step 4: The Spin-Orbit Term $H_{\rm SO}$ \[Back to [top](#toc)\]
$$\label{Hso}$$

From BB2010 Equation (4.18):

\begin{align}
    H_{\rm SO} &= \underbrace{ \frac{ e^{2 \nu - \tilde{\mu} } \left( e^{\tilde{\mu} + \nu} - \tilde{B} \right) \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right) }{ \tilde{B}^{2} \sqrt{Q} \xi^{2} } }_{\rm Term\ 1} + \underbrace{ \frac{ e^{\nu - 2 \tilde{\mu}} }{ \tilde{B}^{2} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} } }_{\rm Term\ 2\ coefficient} \left\{ \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right. \right. \\
    & \left. -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2} + e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( 2 \sqrt{Q} + 1 \right) \left[ \tilde{J} \nu_r \left( {\bf S} \cdot {\bf v} \right) - \nu_{\cos \theta} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B} \\
    & \left. -\tilde{J} \tilde{B}_{r} e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( \sqrt{Q} + 1 \right) \left( {\bf S} \cdot {\bf v} \right) \right\}.
\end{align}

We will denote by Term 2 the expression in $\left\{ \right\}$.  For further information, see [$H_{\rm SO}\ {\rm Term\ 1}$](#HsoTerm1), [$H_{\rm SO}\ {\rm Term\ 2\ coefficient}$](#HsoTerm2coeff), and [$H_{\rm SO}\ {\rm Term\ 1}$](#HsoTerm2).

In [6]:
%%writefile -a SEOBNR/Hamiltonian.txt

Hso = HsoTerm1 + HsoTerm2coeff*HsoTerm2

Appending to SEOBNR/Hamiltonian.txt


<a id='HsoTerm1'></a>
## Step 4a: $H_{\rm SO}$ Term 1 \[Back to [top](#toc)\]
$$\label{HsoTerm1}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1} = \frac{ e^{2 \nu - \tilde{\mu} } \left( e^{\tilde{\mu} + \nu} - \tilde{B} \right) \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right) }{ \tilde{B}^{2} \sqrt{Q} \xi^{2} }
\end{equation}

We divide $H_{\rm SO}\ {\rm Term\ 1}$ into [numerator](#HsoTerm1num) and [denominator](#HsoTerm1den).

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm1 = HsoTerm1num/HsoTerm1den

<a id='HsoTerm1num'></a>

### $H_{\rm SO}$ Term 1 Numerator \[Back to [top](#toc)\]
$$\label{HsoTerm1num}$$

The numerator of $H_{\rm SO}$ Term 1 is

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1\ numerator} = \underbrace{e^{2 \nu - \tilde{\mu}}}_{exp2nuexpmu} \underbrace{ \left( e^{\tilde{\mu} + \nu} - \tilde{B} \right) }_{expmuexpnuminusB} \underbrace{ \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) }_{pdotxir} \underbrace{ \left( {\bf S} \cdot \hat{\bf S}_{\rm Kerr} \right) }_{SdotSkerr}
\end{equation}

We break $H_{\rm SO}$ Term 1 Numerator into four terms: [$e^{2 \nu - \tilde{\mu} }$](#FIXME),  [$e^{\tilde{\mu} + \nu} - \tilde{B}$](#FIXME), [$\hat{\bf p} \cdot \boldsymbol{\xi} r$](#FIXME), and [${\bf S} \cdot \hat{\bf S}_{\rm Kerr}$](#FIXME).

In [ ]:
%%writefile -a SEOBNR/Hamiltonian.txt

HsoTerm1num = exp2nuexpmu*expmuexpnuminusB*pdotxir*SdotSkerr

<a id='HsoTerm1den'></a>

### $H_{\rm SO}$ Term 1 Denominator \[Back to [top](#toc)\]
$$\label{HsoTerm1den}$$

The denominator of $H_{\rm SO}$ Term 1 is

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 1\ denominator} = \tilde{B}^{2} \sqrt{Q} \xi^{2}
\end{equation}

We break $H_{\rm SO}$ Term 1 denominator into three terms:

<a id='HsoTerm2coeff'></a>
## Step 4b: $H_{\rm SO}$ Term 2 Coefficient \[Back to [top](#toc)\]
$$\label{HsoTerm2coeff}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2\ coefficient} = \frac{ e^{\nu - 2 \tilde{\mu}} }{ \tilde{B}^{2} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} }
\end{equation}

We divide $H_{\rm SO}$ Term 2 coefficient into [numerator](#HsoTerm2coeffnum) and [denominator](#HsoTerm2coeffden).

<a id='HsoTerm2'></a>
## Step 4c: $H_{\rm SO}$ Term 2 \[Back to [top](#toc)\]
$$\label{HsoTerm2}$$

We have

\begin{equation}
    H_{\rm SO}\ {\rm Term\ 2} = \left( {\bf S} \cdot \boldsymbol{\xi} \right) \tilde{J} \left[ \mu_r \left( \hat{\bf p} \cdot {\bf v} r \right) \left( \sqrt{Q} + 1 \right) - \mu_{\cos \theta} \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} -\sqrt{Q} \left( \nu_r \left( \hat{\bf p} \cdot {\bf v} r \right) + \left( \mu_{\cos \theta} - \nu_{\cos \theta} \right) \left( \hat{\bf p} \cdot {\bf n} \right) \xi^{2} \right) \right] \tilde{B}^{2} + e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( 2 \sqrt{Q} + 1 \right) \left[ \tilde{J} \nu_r \left( {\bf S} \cdot {\bf v} \right) - \nu_{\cos \theta} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B} -\tilde{J} \tilde{B}_{r} e^{\tilde{\mu} + \nu} \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right) \left( \sqrt{Q} + 1 \right) \left( {\bf S} \cdot {\bf v} \right)
\end{equation}

<a id='Hss'></a>

# Step 5: The Spin-Spin Term $H_{\rm SS}$
$$\label{Hss}$$

From BB2010 Equation (4.19),

\begin{eqnarray}                                                                                                                                                                                            
H_{\rm SS} &=& \omega \left( {\bf S} \cdot {\bf \hat{S}}_{\rm Kerr} \right) + \frac{ e^{-3 \tilde{\mu} -\nu} \tilde{J} \omega_{r} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} } \left\{ -e^{\tilde{\mu} + \nu} \left( {\bf \hat{p}} \cdot {\bf v} r \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right)
\tilde{B} + e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^2 \left( {\bf S}
\cdot {\bf v} \right) \right. \nonumber \\
&& \left. + e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf v} \right)\xi^2 \tilde{B}^{2} + \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left[ \left( {\bf \hat{p}} \cdot {\bf v} r \right)
\left( {\bf S} \cdot {\bf n}\right) - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right)\right] \xi^{2} \tilde{B}^{2} \right\} \nonumber \\
&& + \frac{ e^{-3 \tilde{\mu} - \nu} \omega_{\cos\theta} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} } \left\{ -e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^{2} \left( {\bf S} \cdot {\bf n} \right) + e^{\tilde{\mu} +\nu} \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right) \tilde{B} \right. \nonumber \\
&& \left. + \left[ \left( {\bf S} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf v} r \right)^{2} - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right) \left( {\bf \hat{p}} \cdot {\bf v} r\right) - e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B}^{2} \right\}.
\end{eqnarray}

See [this cell](#HsoHsscommon) for futher definitions.

<a id='HsoHsscommon'></a>

# Step 6: Terms Common to $H_{\rm SO}$ and $H_{\rm SS}$

From BB2010 Equations (5.48)--(5.51):

\begin{align}
\tilde{B} &= BR = \sqrt{ \Delta_{t} } \\
\tilde{B}_r &= B_r R = \frac{ \sqrt{\Delta_{r}} \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{\Delta_{r} \Delta_{r}} } \\
e^{2 \tilde{\mu}} &= e^{2 \mu} R^2 = \Sigma \\
\tilde{J} &= J R = \sqrt{\Delta_{r}} \\
\end{align}

From BB2010 Equations (5.31) and (5.32):

\begin{align}
    \omega = \frac{ \tilde{\omega}_{\rm fd} }{ \Lambda_{t} } \\
    e^{2 \nu} = \frac{ \Delta_{t} \Sigma }{ \Lambda_{t} }.
\end{align}

From BB2010 Equations (5.47a)--(5.47h):

\begin{align}
    B_{r} &= \frac{ \sqrt{ \Delta_{r} } \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{ \Delta_{r} \Delta_{t} } R } \\
    \omega_{r} &= \frac{ -\Lambda^{\prime}_{t} \tilde{\omega}_{\rm fd} + \Lambda_{t} \tilde{\omega}^{\prime}_{\rm fd} }{ \Lambda_{t}^{2} } \\                                                                                                                                      
    \nu_{r} &= \frac{ r }{ \Sigma } + \frac{ \varpi^{2} \left( \varpi^{2} \Delta_{t}^\prime - 4 r \Delta_{t} \right) }{ 2 \Lambda_{t} \Delta_{t} } \\
    \mu_{r} &= \frac{ r }{ \Sigma } - \frac{ 1 }{ \sqrt{ \Delta_{r} } } \\
    B_{\cos \theta} &= 0\\
    \omega_{\cos \theta} =& -\frac{2 a^2 \cos \theta \Delta_{t} \tilde{\omega}_{\rm fd} }{ \Lambda_t^{2} } \\
    \nu_{\cos \theta} &= \frac{a^2 \varpi^{2} \cos \theta \left( \varpi^{2} - \Delta{_t} \right) }{ \Lambda{_t} \Sigma } \\                                                                                                                                                                              
    \mu_{\cos \theta} &= \frac{a^2 \cos \theta }{ \Sigma }                                                                                                                                       
\end{align}

From BB2010 Equations (5.48)--(5.52):

\begin{align}
    \tilde{B} &= BR = \sqrt{ \Delta_{t} } \\
    \tilde{B}_{r} &= B_{r} R = \frac{ \sqrt{ \Delta_{r} } \Delta_{t}^{\prime} - 2 \Delta_{t} }{ 2 \sqrt{ \Delta_{r} \Delta_{t} } } \\
    e^{2 \tilde{\mu}} &= e^{2 \mu} R^{2} = \Sigma \\
    \tilde{J} &= J R = \sqrt{ \Delta_{r} } \\
    Q &= 1 + \frac{ \Delta_{r} \left( \hat{\bf p} \cdot {\bf n} \right)^{2} }{ \Sigma } + \frac{ \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right)^{2} \Sigma }{ \Lambda_{t} \sin^{2} \theta } + \frac{ \left( \hat{\bf p} \cdot {\bf v} r \right)^{2} }{ \Sigma \sin^{2} \theta }
\end{align}

<a id='Hdcoeff'></a>

## $H_{\rm D}$ Coefficient \[Back to [top](#toc)\]
$$\label{Hdcoeff}$$

We have

\begin{equation}
    H_{\rm D}\ {\rm coefficient} = -\frac{ \mu }{ 2 M r^{3} },
\end{equation}

which is written entirely in terms of the coordinate $r$ and the mass terms [$\mu$](#mass_terms) and [$M$](#mass_terms).

<a id='Hdsum'></a>

## $H_{\rm D}$ Sum \[Back to [top](#toc)\]
$$\label{Hdsum}$$

We have

\begin{equation}
    H_{\rm D}\ {\rm sum} = \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j},
\end{equation}

where $\delta^{ij}$ is the Kronecker delta:

\begin{equation}
    \delta_{ij} = \left\{ \begin{array}{cc}
        0, & i \not= j \\
        1, & i = j. \end{array} \right.
\end{equation}

Here ${\bf n}$ denotes normalized coordinates; e.g. if ${\bf R}$ denotes the spherical spacial coordinate vector ${\bf R} = \left( r, \theta, \phi \right)$, then ${\bf n} = \frac{\bf R }{ R }$ where $R = r$ is the magnitude of ${\bf R}$.  

We define [${\bf S}^{*}$](#Hreal_spin_combos) as a combination of spins.

<a id='DeltaSigmaStar'></a>

## $H_{\rm D}$ Term 2b: $\frac{1}{c^{2}} \boldsymbol{\Delta}_{\sigma^{*}}$
$$\label{DeltaSigmaStar}$$

From BB2010 Equation (5.68) we have

\begin{equation}
    \boldsymbol{\Delta}_{\sigma^{*}} = \frac{ \eta }{ 12 } \left[ \frac{ 2M }{ r } \left( 7 \boldsymbol{\sigma}^{*} - 4 \boldsymbol{\sigma} \right) + \left( Q - 1 \right) \left( 3 \boldsymbol{\sigma} + 4 \boldsymbol{\sigma}^{*} \right) - 6 \frac{ \Delta_r }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} \left( 6 \boldsymbol{\sigma} + 5 \boldsymbol{\sigma}^{*} \right) \right]
\end{equation}

where the hatted coordinates are scaled by the mass term [$\mu$](#mass_terms); e.g. $\hat{\bf p} = \frac{ {\bf p} }{ \mu }$.  Here [$\eta$](#mass_terms) is a mass term and $\boldsymbol{\sigma}$, $\boldsymbol{\sigma}^{*}$ are defined [here](#Heff_spin_combos).  The quantity [$Q$](#momentum_terms) is a momentum term.

from BB2010 Equation (5.38)

\begin{equation}
    \Delta_{r} = \Delta_{t} D^{-1}(u);
\end{equation}

from BB2010 Equation (4.5)

\begin{equation}
    \Sigma = r^{2} + a^{2} \cos^{2} \theta;
\end{equation}

from BB2010 Equation (3.23)

\begin{equation}
    \boldsymbol{\xi} = {\bf e}_{Z} \times {\bf N},
\end{equation}

where ${\bf e}_{Z}$ is a Cartesian unit vector in the $z$-direction and ${\bf N} = {\bf n}$ from BB2010 Equation (3.37); from BB2010 Equation (5.37)

\begin{equation}
    \Delta_{t} = r^{2} \left[ A(u) + \frac{ a^{2} }{ M^{2} } u^{2} \right];
\end{equation}

from BB2010 Equation (3.39)

\begin{equation}
    {\bf v} = {\bf n} \times \boldsymbol{\xi};
\end{equation}

from BB2010 Equation (5.42)

\begin{equation}
    D^{-1}(u) = 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3};
\end{equation}

from BB2010 discussion following Equation (5.40)

\begin{equation}
    u = \frac{ M }{ r };
\end{equation}

from BB2010 Equation (4.9)

\begin{equation}
    a = \frac{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }{ M };
\end{equation}

and from BB2010 Equation (5.41)

\begin{equation}
    A(u) = 1 - 2u + 2 \eta u^{3} + \eta \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) u^{4}.
\end{equation}

Oof.

<a id='momentum_terms'></a>

# Step N-5: The Momentum Terms \[Back to [top](#toc)\]
$$\label{momentum_terms}$$

We collect here a variety of terms dependent on the momentum ${\bf p}$.

From BB2010 Equation (5.52)

\begin{equation}
    Q = 1 + \frac{ \Delta_r \left( \hat{\bf p} \cdot {\bf n} \right)^{2} }{ \Sigma } + \frac{ \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right)^{2} \Sigma }{ \Lambda_t \sin^{2} \theta } + \frac{ \left( \hat{\bf p} \cdot {\bf v} r \right)^{2} }{ \Sigma \sin^{2} \theta };
\end{equation}

<a id='alphabetagamma'></a>

# Step N-4: The Metric-Dependent Terms $\alpha$, $\beta^{i}$, and $\gamma^{ij}$ \[Back to [top](#toc)\]
$$\label{alphabetagamma}$$

BB2010 specifies, in Equations (5.44)--(5.46), that for a spacetime metric $g$

\begin{align}
    \alpha &= \frac{ 1 }{ \sqrt{ - g^{tt} }} \\
    \beta^{i} &= \frac{ g^{ti} }{ g^{tt} } \\
    \gamma^{ij} &= g^{ij} - \frac{ g^{ti} g ^{tj} }{ g^{tt} }.
\end{align}

To determine [$H_{\rm real}$](#Hreal) we use the [deformed Kerr metric](#metric).

<a id='Hreal_spin_combos'></a>

# Step N-3: $H_{\rm real}$ Spin Combinations ${\bf S}^{*}$ and ${\bf S}_{\rm Kerr}$ \[Back to [top](#toc)\]
$$\label{Hreal_spin_combos}$$

From BB2010 Equation (5.63):

\begin{equation}
    {\bf S}^{*} = \boldsymbol{\sigma}^{*} + \frac{ 1 }{ c^{2} } \boldsymbol{\Delta}_{\sigma^{*}}.
\end{equation}

We define $\boldsymbol{\sigma}^{*}$ [here](#Heff_spin_combos) and $\boldsymbol{\Delta}_{\sigma^{*}}$ [here](#DeltaSigmaStar).

From BB2010 Equation (5.64):

\begin{equation}
    {\bf S}_{\rm Kerr} = \boldsymbol{\sigma} + \frac{ 1 }{ c^{2} } \boldsymbol{\Delta}_{\sigma}.
\end{equation}

We define $\boldsymbol{\sigma}$ [here](#Heff_spin_combos).  In BB2010 Equation (5.67), $\boldsymbol{\Delta}_{\sigma^{*}} = 0$.

<a id='Heff_spin_combos'></a>

# Step N-2: $H_{\rm eff}$ Spin Combinations $\boldsymbol{\sigma}$ and $\boldsymbol{\sigma}^{*}$ \[Back to [top](#toc)\]
$$\label{Heff_spin_combos}$$

From BB2010 Equation (5.2):

\begin{equation}
    \boldsymbol{\sigma} = {\bf S}_{1} + {\bf S}_{2}.
\end{equation}

From BB2010 Equation (5.3):

\begin{equation}
    \boldsymbol{\sigma}^{*} = \frac{ m_{1} }{ m_{2} } \left( {\bf S}_{1} + {\bf S}_{2} \right).
\end{equation}

These are both written completely in terms of intrinsic physical parameters.

<a id='mass_terms'></a>

# Step N-1: Mass Terms $\mu$, $M$, and $\eta$ \[Back to [top](#toc)\]
$$\label{mass_terms}$$

Each of the terms $\mu$, $M$, and $\eta$ are combinations of the massess $m_{1}$, $m_{2}$ and are defined in the discussion before BB2010 Equation (5.1):

\begin{align}
    M &= m_{1} + m_{2} \\
    \mu &= \frac{ m_{1} m_{2} }{ M } \\
    \eta &= \frac{ \mu }{ M }.
\end{align}

Written completely in terms of the intrinsic physical parameters,

\begin{align}
    \mu &= \frac{ m_{2} m_{2} }{ m_{1} + m_{2} } \\
    \eta &= \frac{ m_{1} m_{2} }{ \left( m_{1} + m_{2} \right)^{2} }.
\end{align}

<a id='metric'></a>

# Step N: The Metric $g$ of Choice \[Back to [top](#toc)\]
$$\label{metric}$$

In BB2010 Equations (5.36a)--(5.36e), we are given the appropriate choice of metric via

\begin{align}
    g^{tt} &= -\frac{ \Lambda_{t} }{ \Delta_{t} \Sigma } \\
    g^{rr} &= \frac{ \Delta_{r} }{ \Sigma } \\
    g^{\theta\theta} &= \frac{ 1 }{ \Sigma } \\
    g^{\phi\phi} &= \frac{ 1 }{ \Lambda_{t} } \left( -\frac{ \tilde{\omega}_{\rm fd}^{2} }{ \Delta_{t} \Sigma } + \frac{ \Sigma }{ \sin^{2} \theta } \right) \\
    g^{t\phi} &= -\frac{ \tilde{\omega}_{\rm fd} }{ \Delta_{t} \Sigma }
\end{align}

From BB2010 Equations (5.71), we have

\begin{equation}
    \Delta_{t} = r^{2} \Delta_{u}(u)
\end{equation}

and from Equations (5.73)--(5.76) we have

\begin{align}
\Delta_{u} &= \bar{\Delta}_u(u) \left[ 1 + \eta \Delta_0 + \eta \log \left( 1 + \Delta_1 u + \Delta_2 u^2 + \Delta_3 u^3 + \Delta_4 u^4 \right) \right] \\
    \bar{\Delta}_u(u) &= \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 2 u }{ \eta K - 1 } + \frac{ 1 }{ \left( \eta K - 1 \right)^{2} }
\end{align}

where $K$ is an adjustable parameter.  The coefficients $\Delta_{0}, \Delta_{1}, ..., \Delta_{4}$ are given in BB2010 Equations (5.77)--(5.81):

\begin{align}
    \Delta_0 &= K \left( \eta K - 2 \right) \\
    \Delta_1 &= -2 \left( \eta K - 1 \right) \left( K + \Delta_0 \right) \\
    \Delta_2 &= \frac{1}{2} \Delta_1 \left( -4 \eta K + \Delta_1 + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_0 \\
    \Delta_3 &= \frac{1}{3} \left[ -\Delta_1^{3} + 3 \left( \eta K - 1 \right) \Delta_1^{2} + 3 \Delta_2 \Delta_1 - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_2 + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_1 \right] \\
    \Delta_4 &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_1^{2} - 2 \Delta_2 \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_1^{4} - 8 \left( \eta K - 1 \right) \Delta_1^{3} - 12 \Delta_2 \Delta_1^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_2 + \Delta_3 \right] \Delta_1 + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^2 \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_2^{2} - 4 \Delta_3 \left( \eta K - 1 \right) \right] \right\}
\end{align}


(5.37)--(5.40) and (5.71)--(5.72),

\begin{align}
    \Delta_{t} &= r^{2} \Delta_{u}(u) \\
    \Delta_{u}(u) &= A(u) + \frac{ a^{2} }{ M^{2} } u^2 \\
    \Delta_{r} &= \Delta_{t} D^{-1}(u) \\
    \Lambda_{t} &= \varpi^{4} - a^{2} \Delta_{t} \sin^{2} \theta \\
    \tilde{\omega}_{\rm fd} &= 2 a M r + \omega_1^{\rm fd} \eta \frac{ a M^{3} }{ r } + \omega_{2}^{\rm fd} \eta \frac{ M a^{3} }{ r }
\end{align}

with $u = \frac{ M }{ r }$ and frame-dragging strength adjustable parameters $\omega_1^{\rm fd}$, $\omega_2^{\rm fd}$.  Finally, from BB2010 Equations (5.41) and (5.42)

\begin{align}
    A(u) &= 1 - 2 u + 2 \eta u^{3} + \eta \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) u^{4} \\
    D^{-1}(u) &= 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3}.
\end{align}

<a id='notes'></a>
# Notes \[Back to [top](#toc)\]
$$\label{notes}$$

1. First, just break down the Hamiltonian as in the paper.  Break up into blocks as appropriate.
2. Use python markdown to write string of input variables:

```python
# Step 0a: declare instrinsic system paramters (masses and spin components)
m1, m2, s1x, s1y, s1z, s2x, s2y, s2z = sp.symbols("m1 m2 s1x s1y s1z s2x s2y s2z",real=True)

```
3. Write terms to files in backwards order!  As in:

    ```python
    #First write is this:
    #%%writefile SEOBNR/Hamiltonian.txt

    ```
   and afterward

    ```python
    #%%writefile -a SEOBNR/Hamiltonian.txt

    ```
   The goal is to split up the terms so that each $\LaTeX$-expression at the bottom of a markdown cell can be easily compared to the "bigstring" expression that is SymPy compatible in a code cell directly below the markdown cell.
4. ${\cal Q}_{4}$ is quartic in $p_{i}$ and is given by Equation (4.34) of [Damour, Jaranowski, and Schaefer](https://arxiv.org/abs/gr-qc/0005034) <font color='red'> (double check this!)</font>

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [7]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-SEOBNR_Documentation.ipynb to latex
[NbConvertApp] Writing 54983 bytes to Tutorial-SEOBNR_Documentation.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
